In [1]:
import _init_paths
from datasets.factory import get_imdb
import visdom
import numpy as np
import cv2

imdb = get_imdb('voc_2007_trainval')

In [2]:
imdb.image_path_at(2018)

'/home/ubuntu/code/visual_learning_and_recognition/hw2/code/data/VOCdevkit2007/VOC2007/JPEGImages/003998.jpg'

In [3]:
a = imdb._load_pascal_annotation('003998')
a

{'boxes': array([[ 11,  59, 499, 306]], dtype=uint16),
 'flipped': False,
 'gt_classes': array([7], dtype=int32),
 'gt_overlaps': <1x21 sparse matrix of type '<type 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'seg_areas': array([121272.], dtype=float32)}

In [4]:
imdb.classes

('aeroplane',
 'bicycle',
 'bird',
 'boat',
 'bottle',
 'bus',
 'car',
 'cat',
 'chair',
 'cow',
 'diningtable',
 'dog',
 'horse',
 'motorbike',
 'person',
 'pottedplant',
 'sheep',
 'sofa',
 'train',
 'tvmonitor')

In [7]:
gt_roidb = imdb.gt_roidb()

voc_2007_trainval gt roidb loaded from /home/ubuntu/code/visual_learning_and_recognition/hw2/code/data/cache/voc_2007_trainval_gt_roidb.pkl


In [15]:
i = 0 
class_to_idx = {}
for cls in imdb.classes:
    class_to_idx[cls] = i
    i += 1
class_to_idx

{'aeroplane': 0,
 'bicycle': 1,
 'bird': 2,
 'boat': 3,
 'bottle': 4,
 'bus': 5,
 'car': 6,
 'cat': 7,
 'chair': 8,
 'cow': 9,
 'diningtable': 10,
 'dog': 11,
 'horse': 12,
 'motorbike': 13,
 'person': 14,
 'pottedplant': 15,
 'sheep': 16,
 'sofa': 17,
 'train': 18,
 'tvmonitor': 19}

In [16]:
print(len(gt_roidb))
    #TODO: return list of (image path, list(+ve class indices)) tuples
print(gt_roidb[5010])
imdb.image_path_at(5010)


5011
{'boxes': array([[ 68,   3, 391, 344]], dtype=uint16), 'gt_classes': array([12], dtype=int32), 'gt_overlaps': <1x21 sparse matrix of type '<type 'numpy.float32'>'
	with 1 stored elements in Compressed Sparse Row format>, 'seg_areas': array([110808.], dtype=float32), 'flipped': False}


'/home/ubuntu/code/visual_learning_and_recognition/hw2/code/data/VOCdevkit2007/VOC2007/JPEGImages/009961.jpg'

In [17]:
def make_dataset(imdb, class_to_idx):
    #TODO: return list of (image path, list(+ve class indices)) tuples
    #You will be using this in IMDBDataset
    gt_roidb = imdb.gt_roidb()
    images = [None]*len(gt_roidb)
    for i in range(len(gt_roidb)):
        path = imdb.image_path_at(i)
        cls = [c for c in gt_roidb[i]['gt_classes']]
        images[i] = (path,cls)
    return images
imgs = make_dataset(imdb, class_to_idx)

voc_2007_trainval gt roidb loaded from /home/ubuntu/code/visual_learning_and_recognition/hw2/code/data/cache/voc_2007_trainval_gt_roidb.pkl


In [18]:
imgs[0]

('/home/ubuntu/code/visual_learning_and_recognition/hw2/code/data/VOCdevkit2007/VOC2007/JPEGImages/000005.jpg',
 [9, 9, 9])

In [7]:
ss_roidb = imdb.selective_search_roidb()

voc_2007_trainval gt roidb loaded from /home/ubuntu/code/visual_learning_and_recognition/hw2/code/data/cache/voc_2007_trainval_gt_roidb.pkl
wrote ss roidb to /home/ubuntu/code/visual_learning_and_recognition/hw2/code/data/cache/voc_2007_trainval_selective_search_roidb.pkl


In [8]:
ss_roidb[2018]

{'boxes': array([[ 11,  59, 499, 306],
        [  0,  63, 499, 336],
        [320, 161, 416, 281],
        ...,
        [369, 288, 412, 364],
        [180, 300, 208, 331],
        [328, 165, 368, 189]], dtype=int32), 'boxscores': array([[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.01060787],
        [0.01053118],
        [0.01023103]]), 'flipped': False, 'gt_classes': array([7, 0, 0, ..., 0, 0, 0], dtype=int32), 'gt_overlaps': <2853x21 sparse matrix of type '<type 'numpy.float32'>'
 	with 2496 stored elements in Compressed Sparse Row format>, 'gt_vec': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32), 'seg_areas': array([121272.,      0.,      0., ...,      0.,      0.,      0.],
       dtype=float32)}

In [28]:
def vis_detections(im, dets):
    """Visual debugging of detections."""
    for i in range(np.minimum(10, dets.shape[0])):
        bbox = tuple(int(np.round(x)) for x in dets[i, :4])
        #score = dets[i, -1]
        
        cv2.rectangle(im, bbox[0:2], bbox[2:4], (0, 204, 0), 2)
        cv2.putText(im, 'rank: %d' % (i), (bbox[0], bbox[1] + 15), cv2.FONT_HERSHEY_PLAIN,
                        1.0, (0, 0, 255), thickness=1)
    return im

dets = ss_roidb[2018]['boxes']
img = cv2.imread(imdb.image_path_at(2018))
res_img = vis_detections(img, dets)

In [31]:
vis = visdom.Visdom(server='http://localhost',port='8097')
vis.text('Task 0, Proposals')
vis.image(res_img.transpose((2,0,1)) )

u'window_361294280ee3ec'

In [27]:
# print(type(res_img))
# res_img.shape

<type 'numpy.ndarray'>


(375, 500, 3)

In [35]:
gt_dets = gt_roidb[2018]['boxes']
print(gt_roidb[2018]['boxes'])
img = cv2.imread(imdb.image_path_at(2018))
gt_res_img = vis_detections(img, gt_dets)
vis.image(gt_res_img.transpose((2,0,1)) )

[[ 11  59 499 306]]


u'window_3612952b9bdff8'

In [24]:
model = torchvision.models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/ubuntu/.torch/models/alexnet-owt-4df8aa71.pth
100.0%


In [27]:
model.features
for param in model.features:
    param.requires_grad = False
# for param in model.parameters():
#     print(param)
    #param.requires_grad = False
    
    
    